# IMPORTS

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, train_test_split
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
from sklearn.cluster import KMeans



## LEITURA DOS DADOS

In [3]:
df = pd.read_csv('../../RawData/train_radiomics_occipital_CONTROL.csv') 
x_train = df.drop(columns=['Transition'])

# Tratamento

### Single Value Colums 

In [4]:
df = df.loc[:, df.nunique() > 1]


#  Analise das colunas que tem menos de 50 valores unicos 
n = df.nunique()
#for col, e in n.items():
#    if e < 50:  
#        print(f"Coluna: {col}, Valores Unicos : {e}")

### AGE BINING  ( secalhar isto era no 2 )

In [5]:

age_bins = [0, 65, 75, 85, 100]
# BINS_SIZER = ['<65', '65-74', '75-84', '85+']
age_labels = [60, 70, 80, 90] # VALOR MEDIO DO BIN 
df['Age'] = pd.cut(df['Age'], bins=age_bins, labels=age_labels).astype(int)

#print(df['Age'])

### Colunas Categoricas 

In [6]:
# Analisar a contagem de valores únicos para cada coluna categórica

# Identificar as colunas categóricas
categorical_columns = df.select_dtypes(include=['object', 'category']).columns

#for col in categorical_columns:
#    print(f"\nColuna: {col}")

#colunas_catagoricas_a_remover = ['ID', 'Image', 'Mask', 'diagnostics_Image-original_Hash', 'diagnostics_Mask-original_Hash'] 

colunas_catagoricas_a_remover = ['ID', 'Image', 'Mask', 'diagnostics_Image-original_Hash', 'diagnostics_Mask-original_Hash', 'diagnostics_Mask-original_BoundingBox', 'diagnostics_Mask-original_CenterOfMassIndex', 'diagnostics_Mask-original_CenterOfMass'] 

# ** Bounding Box
#
# ** as colunas do 'diagnostics_Mask-original_BoundingBox', 'diagnostics_Mask-original_CenterOfMassIndex', 'diagnostics_Mask-original_CenterOfMass'
# ** Deveriam ser retiradas, mas o bounding box pode ser importante para a zona de maior ativação do Alzimeir 
# **  ja a de centro de maxima devem ser muito correlacionados, por isso devem ser retirados mais para a frente 
# *TODO acabei por retirar para correr melhor os modelos, mas analisar se se deve retirar ou nao 

df.drop(columns=colunas_catagoricas_a_remover,axis= 1 , inplace= True)


### Eliminar Transition CN-MCI 

Como este tipo de precisao não vai ser realiza, decidimos retirar todas as linhas com ela relacinadas para nao confundir o nosso modelo

In [7]:
#df = df[df['Transition'] != 'CN-MCI']

### Label Encoding

In [8]:
label_mapping = {
    'CN-CN': 0,
    'AD-AD': 1,
    'MCI-AD': 2,
    'MCI-MCI': 3,
    'CN-MCI' : 4
}

df['Transition'] = df['Transition'].map(label_mapping)

## Correlacao com target 

In [9]:
def remove_highly_correlated_with_target(data, target_column, threshold):
    # Compute the correlation of each column with the target column
    corr_with_target = data.corr()[target_column].abs()
    
    # Identify columns to remove based on correlation with the target
    to_drop = corr_with_target[corr_with_target >= threshold].index
    to_drop = to_drop.drop(target_column)  # Remove target column itself if included
    
    # Drop the identified columns
    data_final = data.drop(columns=to_drop)

    return data_final
print(df.shape)
df = remove_highly_correlated_with_target(df, 'Transition',threshold=0.18)
print(df.shape)

(295, 2026)
(295, 2026)


## Correlacao colunas

In [10]:
def remove_highly_correlated_features(data, threshold):
    # Compute the correlation matrix
    corr_matrix = data.corr().abs()

    # Identify columns to remove based on correlation
    to_drop = set()
    for i in range(len(corr_matrix.columns)):
        for j in range(i + 1, len(corr_matrix.columns)):
            if corr_matrix.iloc[i, j] >= threshold:
                to_drop.add(corr_matrix.columns[j])

    # Drop the identified columns
    data_final = data.drop(columns=to_drop)

    return data_final



print(df.shape)
df = remove_highly_correlated_features(df,0.95)
print(df.shape)

(295, 2026)
(295, 862)


# Models

In [11]:

#experiment = setup(df, target='Transition')

In [12]:
#best = compare_models()

Com o tratamento atual, o pycaret diz que os melhores modelos são :
- Random Forest Classifier
- Extra Trees Classifier
- Gradient Boosting Classifier 
- K Neighbors Classifier
- Extreme Gradient Boosting 


### Aplicação dos modelos (default) ao dataset tratado

In [13]:


X = df.drop('Transition', axis=1)
y = df['Transition']


# Dividir o conjunto de dados em treino e teste com estratificação
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=34, stratify=y)


# Inicializar dicionário para armazenar as pontuações de cross-validation do modelo
model_cross_score = {}


### Funcao Matrix de Confusao 

In [14]:
def plot_confusion_matrix_with_labels(confusion_matrix, label_mapping):
    # Criar um mapeamento inverso
    reverse_label_mapping = {v: k for k, v in label_mapping.items()}
    
    # Aplicar o mapeamento inverso na matriz de confusão
    cm_with_labels = np.zeros_like(confusion_matrix, dtype=object)
    for i in range(confusion_matrix.shape[0]):
        for j in range(confusion_matrix.shape[1]):
            cm_with_labels[i, j] = f"{reverse_label_mapping[i]} (Pred: {reverse_label_mapping[j]})"

    # Criar um DataFrame para facilitar a visualização
    df_cm = pd.DataFrame(confusion_matrix, index=reverse_label_mapping.values(), columns=reverse_label_mapping.values())
    
    # Plotar a matriz de confusão
    plt.figure(figsize=(10, 7))
    sns.heatmap(df_cm, annot=True, fmt="d", cmap="Blues")
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix with Labels')
    plt.show()

#### Random Forest Classifier

In [15]:

rf_model = RandomForestClassifier(n_estimators=100, max_features='sqrt', random_state=34)

rf_score = cross_val_score(rf_model, X_train, y_train, cv=10,scoring='f1_macro')
print(rf_score)

model_cross_score['random_forest'] = rf_score





[0.30995671 0.27736007 0.14973262 0.56944444 0.26436335 0.17156863
 0.71892552 0.14069264 0.4625     0.19191919]


#### Extra Trees Classifier

In [16]:
et_model = ExtraTreesClassifier(criterion='gini', max_depth=20, random_state=34)

et_score = cross_val_score(et_model, X_train, y_train, cv=10,scoring='f1_macro')

print(et_score)

model_cross_score['Extra Trees'] = et_score

[0.31190476 0.25037594 0.21789322 0.44940476 0.42698413 0.17914439
 0.44791667 0.10882353 0.26785714 0.33693609]


#### Gradient Boosting Classifier 

In [17]:
gb_model = GradientBoostingClassifier(learning_rate=0.1, n_estimators=100, random_state=34)

score_cross = cross_val_score(gb_model,X_train,y_train,cv=10,scoring='f1_macro')

print(score_cross)

model_cross_score['Gradient_boosting'] = score_cross



[0.3121345  0.27799423 0.13846154 0.34861111 0.30311355 0.22323232
 0.26764706 0.20625    0.37570208 0.25555556]


#### K Neighbors Classifier

In [18]:
kn_model = KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='auto')
k_night_croos_score = cross_val_score(kn_model, X_train, y_train, cv=10,scoring='f1_macro')

print("Acuracy k night", k_night_croos_score)


model_cross_score['k_night'] = k_night_croos_score




Acuracy k night [0.12698413 0.41614907 0.20192308 0.27579365 0.26450216 0.40396825
 0.27192982 0.17916667 0.3        0.42751623]


#### Extreme Gradient Boosting

In [19]:
xgb_model = XGBClassifier(n_estimators=100, max_depth=4, learning_rate=0.05,colsample_bytree=0.4, subsample=0.8, random_state=34)


score_cross = cross_val_score(xgb_model,X_train,y_train,cv=10,scoring='f1_macro')

print(score_cross)

model_cross_score['XGBoost'] = score_cross



[0.28809524 0.49166667 0.19934641 0.39213564 0.15037594 0.21590909
 0.41066919 0.24846206 0.45753968 0.20133053]


SVC

In [20]:


svc_moldel = SVC(kernel='rbf', random_state=34)


score_cross = cross_val_score(svc_moldel,X_train,y_train,cv=10,scoring='f1_macro')

print(score_cross)

model_cross_score['SVC'] = score_cross



[0.19211823 0.12068966 0.12068966 0.13333333 0.13333333 0.125
 0.125      0.125      0.12068966 0.12068966]


In [21]:
def print_model_scores(model_scores):
    for model, scores in model_scores.items():
        max_score = np.max(scores)
        min_score = np.min(scores)
        diff = max_score - min_score
        avg_score = np.mean(scores)
        
        print(f"{model} (Max: {max_score:.4f}, Min: {min_score:.4f}, Diff: {diff:.4f}):")
        print(f"  Scores: {', '.join([f'{score:.4f}' for score in scores])}")
        print(f"  Average: {avg_score:.4f}")
        print('-' * 50)

# Chamar a função para imprimir o dicionário formatado
print_model_scores(model_cross_score)

random_forest (Max: 0.7189, Min: 0.1407, Diff: 0.5782):
  Scores: 0.3100, 0.2774, 0.1497, 0.5694, 0.2644, 0.1716, 0.7189, 0.1407, 0.4625, 0.1919
  Average: 0.3256
--------------------------------------------------
Extra Trees (Max: 0.4494, Min: 0.1088, Diff: 0.3406):
  Scores: 0.3119, 0.2504, 0.2179, 0.4494, 0.4270, 0.1791, 0.4479, 0.1088, 0.2679, 0.3369
  Average: 0.2997
--------------------------------------------------
Gradient_boosting (Max: 0.3757, Min: 0.1385, Diff: 0.2372):
  Scores: 0.3121, 0.2780, 0.1385, 0.3486, 0.3031, 0.2232, 0.2676, 0.2062, 0.3757, 0.2556
  Average: 0.2709
--------------------------------------------------
k_night (Max: 0.4275, Min: 0.1270, Diff: 0.3005):
  Scores: 0.1270, 0.4161, 0.2019, 0.2758, 0.2645, 0.4040, 0.2719, 0.1792, 0.3000, 0.4275
  Average: 0.2868
--------------------------------------------------
XGBoost (Max: 0.4917, Min: 0.1504, Diff: 0.3413):
  Scores: 0.2881, 0.4917, 0.1993, 0.3921, 0.1504, 0.2159, 0.4107, 0.2485, 0.4575, 0.2013
  Average